In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2

from skimage.io import imread
from skimage.segmentation import chan_vese, watershed
from skimage.draw import ellipse
from scipy import ndimage as ndi
from sklearn.cluster import KMeans
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

1. Separating the masks from the dataset 

In [ ]:
train_directory = './Train/Train'
test_directory = './Test/Test'

In [ ]:
train_images_filenames = os.listdir(train_directory)
test_images_filenames = os.listdir(test_directory)

train_images_filenames = sorted(train_images_filenames)
test_images_filenames = sorted(test_images_filenames)

train_images = []
train_masks = []
test_images = []
test_masks = []

for image in train_images_filenames:
    if image.endswith('_seg.png'):
        # Save the name without the _seg.png and the image
        name = image[:-8]
        image_name = name + '.jpg'
        mask_image = imread(train_directory + '/' + image)
        image_image = imread(train_directory + '/' + image_name)
        train_masks.append([name, mask_image])
        train_images.append([name, image_image])
   
for image in test_images_filenames:
    if image.endswith('_seg.png'):
        # Save the name without the _seg.png and the image
        name = image[:-8]
        image_name = name + '.jpg'
        mask_image = imread(test_directory + '/' + image)
        image_image = imread(test_directory + '/' + image_name)
        test_masks.append([name, mask_image])
        test_images.append([name, image_image])

In [ ]:
# Create folders to store the exported images
test_export_folder = './Exported_Images/Test'
train_export_folder = './Exported_Images/Train'
os.makedirs(test_export_folder, exist_ok=True)
os.makedirs(train_export_folder, exist_ok=True)

# Export test images
for image in test_images:
    name = image[0]
    image_data = image[1]
    image_path = os.path.join(test_export_folder, f'{name}.jpg')
    cv2.imwrite(image_path, image_data)

# Export train images
for image in train_images:
    name = image[0]
    image_data = image[1]
    image_path = os.path.join(train_export_folder, f'{name}.jpg')
    cv2.imwrite(image_path, image_data)


In [ ]:
# Create folders to store the exported images
test_export_folder = './Exported_Images/Test_Masks'
train_export_folder = './Exported_Images/Train_Masks'
os.makedirs(test_export_folder, exist_ok=True)
os.makedirs(train_export_folder, exist_ok=True)

# Export test images
for image in test_masks:
    name = image[0]
    image_data = image[1]
    image_path = os.path.join(test_export_folder, f'{name}.jpg')
    cv2.imwrite(image_path, image_data)

# Export train images
for image in train_masks:
    name = image[0]
    image_data = image[1]
    image_path = os.path.join(train_export_folder, f'{name}.jpg')
    cv2.imwrite(image_path, image_data)

In [3]:
data_labels = pd.read_csv('metadataTrain.csv')

In [5]:


# Assuming data_labels is a pandas DataFrame
selected_data = data_labels[['ID', 'CLASS']]

# Export to csv
selected_data.to_csv('image_label.csv', index=False)


In [ ]:
test_export_folder = './Exported_Images/Test_Masks'
train_export_folder = './Exported_Images/Train_Masks'
os.makedirs(test_export_folder, exist_ok=True)
os.makedirs(train_export_folder, exist_ok=True)

# Export test images
for image in test_masks:
    name = image[0]
    image_data = image[1]
    image_path = os.path.join(test_export_folder, f'{name}.jpg')
    cv2.imwrite(image_path, image_data)

# Export train images
for image in train_masks:
    name = image[0]
    image_data = image[1]
    image_path = os.path.join(train_export_folder, f'{name}.jpg')
    cv2.imwrite(image_path, image_data)

2. SegNET

In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size=(3, 3), strides=(1, 1), padding='same'):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding=padding)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def upsampling_block(inputs, filters, kernel_size=(3, 3), strides=(2, 2), padding='same'):
    x = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    return x

def segnet(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Encoder
    conv1 = conv_block(inputs, 64)
    conv2 = conv_block(conv1, 64)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool1, 128)
    conv4 = conv_block(conv3, 128)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = conv_block(pool2, 256)
    conv6 = conv_block(conv5, 256)
    conv7 = conv_block(conv6, 256)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv7)

    conv8 = conv_block(pool3, 512)
    conv9 = conv_block(conv8, 512)
    conv10 = conv_block(conv9, 512)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv10)

    # Decoder
    upsample1 = upsampling_block(pool4, 512)
    conv11 = conv_block(upsample1, 512)
    conv12 = conv_block(conv11, 512)
    conv13 = conv_block(conv12, 512)

    upsample2 = upsampling_block(conv13, 256)
    conv14 = conv_block(upsample2, 256)
    conv15 = conv_block(conv14, 256)

    upsample3 = upsampling_block(conv15, 128)
    conv16 = conv_block(upsample3, 128)

    upsample4 = upsampling_block(conv16, 64)
    conv17 = conv_block(upsample4, 64)

    # Output
    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(conv17)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model

# Example usage:
input_shape = (256, 256, 3)  # Example input shape
num_classes = 10  # Example number of classes
model = segnet(input_shape, num_classes)
model.summary()




In [1]:
#############AUXILIAR CODE IGNORE IT#################
import cv2
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
from skimage.segmentation import flood_fill
import skimage.morphology as morpho
from matplotlib import pyplot as plt
from numpy.lib.function_base import median
import numpy as np

def remove_skin_hair(image, kernel_size):
    hair_kernel_size = 47
    median_blur_image = cv2.medianBlur(image, hair_kernel_size)
    only_hair = cv2.subtract(median_blur_image, image)

    _, mask = cv2.threshold(only_hair, int(0.02*255),255,cv2.THRESH_BINARY)
    kernel = morpho.disk(2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)
    # Create a copy of the input image to store the inpainted result
    inpainted_image = image.copy()

    # Get the coordinates of the noisy regions from the binary mask
    noisy_coords = np.where(mask > 0)

    for y, x in zip(*noisy_coords):
        y_start, y_end = max(0, y - kernel_size), min(image.shape[0], y + kernel_size + 1)
        x_start, x_end = max(0, x - kernel_size), min(image.shape[1], x + kernel_size + 1)

        neighborhood = image[y_start:y_end, x_start:x_end]

        if neighborhood.size > 0:
            inpainted_image[y, x] = np.median(neighborhood)

    return inpainted_image

def find_good_region(image):
  _, good_region = cv2.threshold(image, 50, 255, cv2.THRESH_BINARY_INV)
  kernel = morpho.disk(5)

  good_region = cv2.morphologyEx(good_region, cv2.MORPH_OPEN, kernel)
  good_region = cv2.morphologyEx(good_region, cv2.MORPH_CLOSE, kernel)

  flag_painted = 0

  if(good_region[0,0]==255):
    regions = flood_fill(good_region, seed_point=(0,0), new_value=128)
    flag_painted = 1
  if(good_region[image.shape[0]-1,image.shape[1]-1]==255):
    regions = flood_fill(regions, seed_point=(image.shape[0]-1,image.shape[1]-1), new_value=128)
    flag_painted = 1
  if(good_region[0,image.shape[1]-1]==255):
    regions = flood_fill(regions, seed_point=(0,image.shape[1]-1), new_value=128)
    flag_painted = 1
  if(good_region[image.shape[0]-1,0]==255):
    regions = flood_fill(regions, seed_point=(image.shape[0]-1,0), new_value=128)
    flag_painted = 1

  if(flag_painted == 0):
    return np.full_like(image, 255, dtype ='uint8')

  good_region = (255*(regions == 128)).astype('uint8')

  kernel = morpho.disk(30)
  good_region = cv2.morphologyEx(good_region, cv2.MORPH_DILATE, kernel)
  good_region = 255 - good_region
  return good_region

def segmentation(inpainted, good_region):
  inner_circle_values = inpainted[(good_region == 255)]

  otsu_threshold, _ = cv2.threshold(inner_circle_values, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
  #print("Obtained threshold: ", otsu_threshold)

  segmentation_mask = np.zeros_like(inpainted)
  segmentation_mask = np.where(good_region == 0, 0, segmentation_mask)
  segmentation_mask = np.where((good_region != 0) & (inpainted < otsu_threshold), 255, segmentation_mask)

  #selecting only the region with the biggest area
  kernel = morpho.disk(20)
  segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_OPEN, kernel)
  kernel = morpho.disk(30)
  segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_DILATE, kernel)
  kernel = morpho.disk(85)
  segmentation_mask = cv2.morphologyEx(segmentation_mask, cv2.MORPH_CLOSE, kernel)

  if(np.all(segmentation_mask == 0)): #check if the mask is null everywhere
    return segmentation_mask

  numlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(segmentation_mask)
  matrix = np.column_stack((range(numlabels), stats[:,4]))
  sorted_matrix = sorted(matrix, key=lambda row: row[1], reverse=True)

  #draw the biggest contour
  final_mask = (255*(labels == sorted_matrix[1][0])).astype('uint8')

  return final_mask

def DICE_COE(mask_true, mask_prevision):
    #mask_prevision = 255 - mask_prevision #invert the result mask
    intersect = np.sum(mask_true*mask_prevision)
    fsum = np.sum(mask_true>0)
    ssum = np.sum(mask_prevision>0)
    dice = (2 * intersect ) / (fsum + ssum)
    dice = round(dice, 3)
    return dice

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:

dice_scores = []

for image in train_images:
    image_name = image[0]
    image_data = image[1]
    
    # Load the corresponding mask image
    mask_path = f"./Exported_Images/Train_Masks/{image_name}.jpg"
    mask_image = imread(mask_path)
    
    img = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    inpainted = remove_skin_hair(img, 40)
    good_region = find_good_region(inpainted)
    final_mask = segmentation(inpainted, good_region)
    
    
    # Compute the dice score
    dice_score = DICE_COE(final_mask.flatten(), mask_image.flatten())
    dice_scores.append(dice_score)

average_dice_score = np.mean(dice_scores)
print(f"Average Dice score for images in Exported_Images/Train: {average_dice_score}")


KeyboardInterrupt: 